In [0]:
import pandas as pd

URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = "Erro ao carregar dados de treino"
erro_teste = "Erro ao carregar dados de teste"
erro_desafioqt = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()

In [0]:
#Exemplo de classificação com Dummy
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

modelo_dummy = DummyRegressor()
modelo_dummy.fit(X_treino, Y_treino)
dummy_predicoes = modelo_dummy.predict(X_teste)

avaliacao_dummy = mean_squared_error(Y_teste, dummy_predicoes)

print(f"Minha avaliação nos dados de teste foi de {avaliacao_dummy}")

Minha avaliação nos dados de teste foi de 5219.286870186777


In [0]:
#aplicando o pre-processamento
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_treino)  # Don't cheat - fit only on training data
X_treino = scaler.transform(X_treino)
X_teste = scaler.transform(X_teste)  # apply same transformation to test data

In [0]:
#aplicando o modelo
#valor a bater 2044.2661201991486
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LassoCV, BayesianRidge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor

estimators = [
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Lasso', LassoCV()),
    ('Gradient Boosting', HistGradientBoostingRegressor(random_state=0)),
    ('Gradient', GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, n_estimators=150, random_state=0)), #
    ('RidgeCV',RidgeCV()),
    ('SGD', SGDRegressor(alpha=0.01,random_state=1, learning_rate='adaptive', warm_start=True, tol=1e-7, eta0=0.000001)), #
    ('LinearSVR', LinearSVR()),
    ('Bayesian', BayesianRidge(verbose=True))]

stacking_regressor = StackingRegressor(
    estimators=estimators, final_estimator=ElasticNetCV()
)
modelo = stacking_regressor
modelo.fit(X_treino, Y_treino)
predicoes_modelo= modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, predicoes_modelo)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")
print(modelo.score(X_teste, Y_teste) * 100)
print(predicoes_modelo[:5].round(1))
print(Y_teste[:5])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Convergence after  1  iterations


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Convergence after  1  iterations
Convergence after  1  iterations
Convergence after  1  iterations
Convergence after  1  iterations
Convergence after  1  iterations
Minha avaliação nos dados de teste foi de 2044.2661201991486
60.83029902359487
[529.5 457.8 649.1 513.1 553.1]
[550.  406.3 652.3 469.1 580.5]
